In [1]:
import pandas as pd
import numpy as np
import glob
import pydicom
import cv2
import os, os.path as osp

from scipy.ndimage.interpolation import zoom
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pylab as plt
import os
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

# import cv2
import glob
import pickle

In [2]:
# from IPython.display import FileLink
# FileLink(r'./dd.pkl')

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

device

device(type='cuda')

In [4]:
!ls ../input/rsna-str-pulmonary-embolism-detection/

sample_submission.csv  test  test.csv  train  train.csv


In [5]:
sample_df = pd.read_csv('../input/rsna-str-pulmonary-embolism-detection/sample_submission.csv')
df = pd.read_csv('../input/rsna-str-pulmonary-embolism-detection/test.csv')

dicom_folders = list(('../input/rsna-str-pulmonary-embolism-detection/test/' + df.StudyInstanceUID + '/'+ df.SeriesInstanceUID).unique())

In [6]:
len(dicom_folders)

650

In [7]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [8]:
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

test_ds_trans = transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.Grayscale(num_output_channels=3),
                                transforms.Resize(224),
                                transforms.ToTensor(),
                                
#                                 transforms.CenterCrop(224),
                                normalize
])

invTrans = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.229, 1/0.224, 1/0.225 ]),
                                transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ],
                                                     std = [ 1., 1., 1. ]),
                               ])

In [9]:
sample_df

,id,label
0,df06fad17bc3_negative_exam_for_pe,0.5
1,c8039e7f9e63_negative_exam_for_pe,0.5
2,761f6f1a9f5b_negative_exam_for_pe,0.5
3,c8db5b1f6b56_negative_exam_for_pe,0.5
4,462e805da1f1_negative_exam_for_pe,0.5
...,...,...
152698,5f34e0c61c00,0.5
152699,ccaa309b60da,0.5
152700,a274c8d0916e,0.5
152701,a702de2c99c6,0.5


In [10]:
def load_dicom_array(f):
    dicom_files = glob.glob(osp.join(f, '*.dcm'))
    dicoms = [pydicom.dcmread(d) for d in dicom_files]
    M = float(dicoms[0].RescaleSlope)
    B = float(dicoms[0].RescaleIntercept)
    # Assume all images are axial
    z_pos = [float(d.ImagePositionPatient[-1]) for d in dicoms]
    dicoms = np.asarray([d.pixel_array for d in dicoms])
    dicoms = dicoms[np.argsort(z_pos)]
    dicoms = dicoms * M
    dicoms = dicoms + B
    return dicoms, np.asarray(dicom_files)[np.argsort(z_pos)]


def window(img, WL=50, WW=350):
    upper, lower = WL+WW//2, WL-WW//2
    X = np.clip(img.copy(), lower, upper)
    X = X - np.min(X)
    X = X / np.max(X)
    X = (X*255.0).astype('uint8')
    return X


def save_array(X, save_dir, file_names):
    for ind, img in enumerate(X):
        savefile = osp.join(save_dir, file_names[ind])
        if not osp.exists(osp.dirname(savefile)): 
            os.makedirs(osp.dirname(savefile))
        _ = cv2.imwrite(osp.join(save_dir, file_names[ind]), img)


def edit_filenames(files):
    dicoms = [f"{ind:04d}_{f.split('/')[-1].replace('dcm','jpg')}" for ind,f in enumerate(files)]
    series = ['/'.join(f.split('/')[-3:-1]) for f in files]
#     print(files[0], dicoms[0], series[0])
    return [osp.join(s,d) for s,d in zip(series, dicoms)]


class Lungs(Dataset):
    def __init__(self, dicom_folders):
        self.dicom_folders = dicom_folders
#         self.trans = trans
    def __len__(self): return len(self.dicom_folders)
    def get(self, i):
        return load_dicom_array(self.dicom_folders[i])
    def __getitem__(self, i):
        try:
            return self.get(i)
        except Exception as e:
            print(e)
            return None


# SAVEDIR = '../../data/train-jpegs/'
MAX_LENGTH = 256.

dset = Lungs(dicom_folders)
loader = DataLoader(dset, batch_size=1, shuffle=False, num_workers=0, collate_fn=lambda x: x)

In [11]:
# mkdir ./test_jpegs

In [12]:
# SAVEDIR = './test_jpegs'

In [13]:
# predictions = {}

In [14]:
resnet101 = models.resnet50()
fc_feat_size = resnet101.fc.in_features
resnet101.fc.in_features, resnet101.fc.out_features

(2048, 1000)

In [15]:
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
    def __forward__(self, x):
        x = x.view(x.size(0), -1)
        return x
    
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x

In [16]:
resnet101.avgpool = Identity()
resnet101.fc = Identity()

In [17]:
def weights_init(m):
    classname = m.__class__.__name__
    for l in m.modules():
        if isinstance(l, nn.Conv2d):
            torch.nn.init.uniform_(l.weight)
        elif isinstance(l, nn.BatchNorm2d):
            torch.nn.init.uniform_(l.weight)
        elif isinstance(l, nn.Linear):
            torch.nn.init.xavier_normal_(l.weight)
        
class MultiTaskHead(nn.Module):
    def __init__(self):
        super(MultiTaskHead, self).__init__()
        self.m = nn.Sequential(
                nn.Conv2d(2048, 512, kernel_size=(1,1), stride=(1,1), bias=False), 
                nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
                nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                nn.Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False),
                nn.BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                nn.AdaptiveAvgPool2d(output_size=(1,1)),
        )
        self.l = nn.Sequential(
                nn.Linear(fc_feat_size, 1, bias=True), 
#                 nn.ReLU(),
#                 nn.Linear(512, 1), 
                nn.Sigmoid()
        )
        
    def forward(self, x):
        x = x.view(x.shape[0], 2048, int(np.sqrt(x.shape[1]/2048)), int(np.sqrt(x.shape[1]/2048)))
        x = self.m(x)
        x = self.l(x.squeeze())
        return x

class WrapperModel(nn.Module):
    def __init__(self, pretrained_model):
        super(WrapperModel, self).__init__()
        self.stage = 'train'
        
        self.backbone = pretrained_model
        '''
        image-level feature
        '''
        self.linear_pe_present_on_image = MultiTaskHead()
        weights_init(self.linear_pe_present_on_image)
        '''
        exam-level features
        '''
        self.linear_negative_exam_for_pe = MultiTaskHead()
        weights_init(self.linear_negative_exam_for_pe)
        
        self.linear_indeterminate = MultiTaskHead()
        weights_init(self.linear_indeterminate)
        
        self.linear_chronic_pe = MultiTaskHead()
        weights_init(self.linear_chronic_pe)
        
        self.linear_acute_and_chronic_pe = MultiTaskHead()
        weights_init(self.linear_acute_and_chronic_pe)
        
        self.linear_central_pe = MultiTaskHead()
        weights_init(self.linear_central_pe)
        
        self.linear_leftsided_pe = MultiTaskHead()
        weights_init(self.linear_leftsided_pe)
        
        self.linear_rightsided_pe = MultiTaskHead()
        weights_init(self.linear_rightsided_pe)
        
        self.linear_rv_lv_ratio_gte_1 = MultiTaskHead()
        weights_init(self.linear_rv_lv_ratio_gte_1)
        
        self.linear_rv_lv_ratio_lt_1 = MultiTaskHead()
        weights_init(self.linear_rv_lv_ratio_lt_1)
        
        '''
        informational features
        '''
        self.linear_qa_motion = MultiTaskHead()
        weights_init(self.linear_qa_motion)
        
        self.linear_qa_contrast = MultiTaskHead()
        weights_init(self.linear_qa_contrast)
                
        self.linear_true_filling_defect_not_pe = MultiTaskHead()
        weights_init(self.linear_true_filling_defect_not_pe)
        
        self.linear_flow_artifact = MultiTaskHead()
        weights_init(self.linear_flow_artifact)

    def forward(self, x):
#         if self.stage == 'train':
#             x, _ = self.backbone(x)
#         else: 
        x = self.backbone(x)
        
#         print('x1 .shape', x1.shape)
        x_pe_present_on_image = self.linear_pe_present_on_image(x)
        
        x_negative_exam_for_pe = self.linear_negative_exam_for_pe(x)
        x_indeterminate = self.linear_indeterminate(x)
        x_chronic_pe = self.linear_chronic_pe(x)
        x_acute_and_chronic_pe = self.linear_acute_and_chronic_pe(x)
        x_central_pe = self.linear_central_pe(x)
        x_leftsided_pe = self.linear_leftsided_pe(x)
        x_rightsided_pe = self.linear_rightsided_pe(x)
        x_rv_lv_ratio_gte_1 = self.linear_rv_lv_ratio_gte_1(x)
        x_rv_lv_ratio_lt_1 = self.linear_rv_lv_ratio_lt_1(x)
        
        x_qa_motion = self.linear_qa_motion(x)
        x_qa_contrast = self.linear_qa_contrast(x)
        x_true_filling_defect_not_pe = self.linear_true_filling_defect_not_pe(x)
        x_flow_artifact = self.linear_flow_artifact(x)
        
#         ys = []
#         for y_pred in [ x_pe_present_on_image, x_negative_exam_for_pe, x_indeterminate, x_chronic_pe, x_acute_and_chronic_pe, x_central_pe, x_leftsided_pe, x_rightsided_pe, x_rv_lv_ratio_gte_1, x_rv_lv_ratio_lt_1, x_qa_motion, x_qa_contrast, x_true_filling_defect_not_pe, x_flow_artifact]:
#             ys.append(y_pred)
        ys = [x_pe_present_on_image, x_indeterminate, x_chronic_pe, x_acute_and_chronic_pe, x_central_pe, x_leftsided_pe, x_rightsided_pe, x_rv_lv_ratio_gte_1, x_rv_lv_ratio_lt_1, x_qa_motion, x_qa_contrast, x_true_filling_defect_not_pe, x_flow_artifact]
#         for y_pred in [ x_pe_present_on_image, x_indeterminate, x_chronic_pe, x_acute_and_chronic_pe, x_central_pe, x_leftsided_pe, x_rightsided_pe, x_rv_lv_ratio_gte_1, x_rv_lv_ratio_lt_1, x_qa_motion, x_qa_contrast, x_true_filling_defect_not_pe, x_flow_artifact]:
#             ys.append(y_pred)
        if self.stage == 'eval':
#             print(len(ys), ys[0].shape)
#             print(torch.cat(ys, 1).shape)
            return torch.cat(ys, 1).cpu().numpy()
        else:
            return torch.cat(ys, 1)
            

model = WrapperModel(resnet101).to(device)

In [18]:
ls ../input/resnet-trained

resnet_2_9011964525914555_0_9047383336849932__0_3563135365031113_0_2710275558108921


In [19]:
model.load_state_dict(torch.load('../input/resnet-trained/resnet_2_9011964525914555_0_9047383336849932__0_3563135365031113_0_2710275558108921'))
model.stage='eval'
model.eval()

WrapperModel(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
    

In [20]:
with torch.no_grad():
    for i, data in enumerate(tqdm(loader, total=len(loader))):
        data = data[0]
        if type(data) == type(None): continue
#         try:
        image, files = data
        # Windows from https://pubs.rsna.org/doi/pdf/10.1148/rg.245045008
        image_lung = np.expand_dims(window(image, WL=-600, WW=1500), axis=3)
        image_mediastinal = np.expand_dims(window(image, WL=40, WW=400), axis=3)
        image_pe_specific = np.expand_dims(window(image, WL=100, WW=700), axis=3)
        image = np.concatenate([image_mediastinal, image_pe_specific, image_lung], axis=3)
        rat = MAX_LENGTH / np.max(image.shape[1:])
        image = zoom(image, [1.,rat,rat,1.], prefilter=False, order=1)

        images = [test_ds_trans(x_) for x_ in image]
        x = torch.stack(images).to(device)
        y_pred = model(x)
        
        study_instance = files[0].split('/')[-3]
        
        c = [f.split('/')[-1][:-4] for f in files]
        indices = [c.index(f) for f in sample_df[sample_df['id'].isin(c)].id.values.tolist()]
        '''
        filling image level predictions
        '''
        sample_df.at[sample_df['id'].isin(c), 'label'] = ((y_pred[indices,:]>=0.5).sum(1)>0).astype(int).tolist()
        '''
        filling study level predictions
        '''
        study_results = ((y_pred[:,[7,8,5,2,6,3,4,1]] >=0.5).sum(0) > 0).astype(int).tolist()
        negative_ex = 1 - max(study_results)
        sample_df.loc[sample_df.index[sample_df['id'].str.contains(study_instance)], 'label'] = [negative_ex] + study_results
        

100%|██████████| 650/650 [2:35:00<00:00, 14.31s/it]  


In [21]:
save_obj(sample_df, './sample_df')
from IPython.display import FileLink
FileLink(r'./sample_df.pkl')

/kaggle/working/sample_df.pkl

In [23]:
sample_df

,id,label
0,df06fad17bc3_negative_exam_for_pe,0.0
1,c8039e7f9e63_negative_exam_for_pe,1.0
2,761f6f1a9f5b_negative_exam_for_pe,0.0
3,c8db5b1f6b56_negative_exam_for_pe,0.0
4,462e805da1f1_negative_exam_for_pe,1.0
...,...,...
152698,5f34e0c61c00,0.0
152699,ccaa309b60da,0.0
152700,a274c8d0916e,0.0
152701,a702de2c99c6,0.0


In [37]:
sample_df.to_csv('submission.csv', index = False)

In [31]:
(sample_df.label < 0).any(), (sample_df.label > 1).any()

(False, False)

In [34]:
sample_df.isnull().values.any()

False

In [38]:
!ls

__notebook_source__.ipynb  sample_df.pkl  submission.csv


In [35]:
# f = open('submission.csv', 'w')
# f.write('id,label\n')